In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import requests, re, json ,pprint
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import urllib.request
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
from urllib.parse import urlparse
from time import sleep
import datetime
import pytz # new import
new_timezone = pytz.timezone("US/Eastern")
import spacy
import en_core_web_sm
import en_core_web_lg
import en_vectors_web_lg
import en_core_web_md
import csv
from dateparser.search import search_dates
import arrow
import datetime
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import sys
csv.field_size_limit(sys.maxsize)

131072

In [2]:
driver_path='./chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')   
driver = webdriver.Chrome(driver_path,options=options)
# driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

In [3]:
def reload_test_site():
    my_url = 'https://www.youtube.com/watch?v=A1tqTQkC36A&amp;lc=Ugw0nq07e-G5JyUsl094AaABAg'
    driver.get(my_url)
    sleep(1)

In [4]:
def get_driver(driver,my_url):
#     my_url = 'https://www.youtube.com/watch?v=A1tqTQkC36A&amp;lc=Ugw0nq07e-G5JyUsl094AaABAg'
    driver.get(my_url)
    return driver
#     sleep(1)

## Define User ID

In [5]:
def extract_transcript(driver):
    sleep(2)
    driver.implicitly_wait(10)
    test = driver.find_element_by_xpath("//button[@aria-label='More actions']");
    test.click()
    driver.implicitly_wait(10)
    test_out = driver.find_element_by_xpath('//yt-formatted-string[text() ="Open transcript"]')
    test_out.click()
    driver.implicitly_wait(10)
    test_out = driver.find_element_by_xpath('//div[@id="body"]/ytd-transcript-body-renderer')
    test_body = test_out.text.encode('utf-8')    
    
    return test_body.decode('utf-8','ignore')

## Extract from List of Videos

In [20]:
#get file from cloud, skip if not necessary
!gsutil cp gs://ytscrape/$csv_input ./
!gsutil cp gs://yttranscript/$csv_output ./

Copying gs://ytscrape/batch21_taylor.csv...
| [1 files][113.8 MiB/113.8 MiB]                                                
Operation completed over 1 objects/113.8 MiB.                                    
Copying gs://yttranscript/transcript_batch21_taylor.csv...
- [1 files][133.8 KiB/133.8 KiB]                                                
Operation completed over 1 objects/133.8 KiB.                                    


In [92]:
#define filename
csv_input = 'tryguys_underwear.csv'
csv_output = 'transcript_' + csv_input

In [93]:
%%bash -s "$csv_input" 
echo "$1"
# head "$1"
echo video_url > temp
bash ./extract_unique_url.sh "$1" >> temp

tryguys_surgery.csv


In [94]:
filenames = pd.read_csv(filepath_or_buffer='./temp',header='infer')
vid_id_df =filenames.video_url.unique()

In [95]:
del filenames
display(len(vid_id_df))

1

In [58]:
ctr = 0
rows = 0

csv.field_size_limit(sys.maxsize)
with open(csv_output) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if 'https' in row[0]:
            rows += 1
            ctr += row[0].count('https')

print('videos: {}, rows: {}'.format(ctr, rows))

videos: 0, rows: 0


In [96]:
driver_path='./chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')   
driver = webdriver.Chrome(driver_path,options=options)
# driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

with open(csv_output, 'w', newline='') as csvfile:
    fieldnames = ['url', 'transcript']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,
            delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for urls in vid_id_df:
        try:
            driver.get(urls)
            text = extract_transcript(driver)
            x = {"url":urls,"transcript":text}
            writer.writerow(x)
        except:
            pass
print('done')

done


In [58]:
!gsutil mv $csv_output gs://yttranscript/

Copying file://transcript_batch8_maldonado.csv [Content-Type=text/csv]...
Removing file://transcript_batch8_maldonado.csv...                              

Operation completed over 1 objects/2.7 MiB.                                      


array([], dtype=object)

In [50]:
!gsutil cp batch24* gs://ytscrape

Copying file://batch24_holiday.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][191.9 MiB/191.9 MiB]                                                
Operation completed over 1 objects/191.9 MiB.                                    


In [70]:
!gsutil cp transcript_batch34* gs://yttranscript

Copying file://transcript_batch34_aylaststewart.csv [Content-Type=text/csv]...
- [1 files][  4.3 MiB/  4.3 MiB]                                                
Operation completed over 1 objects/4.3 MiB.                                      


In [46]:
!gsutil cp combine.sql gs://ytscrape

Copying file://combine.sql [Content-Type=application/x-sql]...
/ [1 files][  597.0 B/  597.0 B]                                                
Operation completed over 1 objects/597.0 B.                                      


In [94]:
!gsutil mv batch5* gs://ytscrape

Copying file://batch50_mouthybuddha-Copy1.csv [Content-Type=text/csv]...
Removing file://batch50_mouthybuddha-Copy1.csv...                               
Copying file://batch50_mouthybuddha.csv [Content-Type=text/csv]...
Removing file://batch50_mouthybuddha.csv...                                     
Copying file://batch50_styxhexen.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow th